In [1]:
from pylab import *

In [51]:
def Mat_A_inv(Nx,Ny,r):
    # Ojo Nx,Ny se refiere a los pintos interiores
    N_tot = Nx*Ny
    A = zeros( (N_tot,N_tot) )
    dd , m_r= 1+4*r , -r
    # Psi_ij -> i renglones o y, j columnas o x
    for i in range(Ny-1):
        for j in range(Nx-1):
            A[Ny*i+j,Ny*i+j] = 1 #dd
            A[Ny*i+j,Ny*i+j+1] = 2 #m_r
            A[Ny*i+j+1,Ny*i+j] = 3 #m_r
            A[Ny*i+j,Ny*(i+1)+j] = 4 #m_r
            A[Ny*(i+1)+j,Ny*i+j] = 5 #m_r
    for j in range(Nx-1):
        A[Ny*(Ny-1)+j,Ny*(Ny-1)+j] = 1 #dd
        A[Ny*(Ny-1)+j,Ny*(Ny-1)+j+1] = 2 #m_r
        A[Ny*(Ny-1)+j+1,Ny*(Ny-1)+j] = 3 #m_r
    for i in range(Ny-1):
        A[Ny*i+Nx-1,Ny*i+Nx-1] = 1 #m_r
        A[Ny*i+Nx-1,Ny*(i+1)+Nx-1] = 4 #m_r
        A[Ny*(i+1)+Nx-1,Ny*i+Nx-1] = 5 #m_r
    A[Ny*(Ny-1)+Nx-1,Ny*(Ny-1)+Nx-1] = 1#dd
    return A

In [52]:
Mat_A_inv(4,4,1)

array([[1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 3., 1., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0.],
       [0., 5., 0., 0., 3., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0.],
       [0., 0., 5., 0., 0., 3., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0.],
       [0., 0., 0., 5., 0., 0., 3., 1., 0., 0., 0., 4., 0., 0., 0., 0.],
       [0., 0., 0., 0., 5., 0., 0., 0., 1., 2., 0., 0., 4., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0., 3., 1., 2., 0., 0., 4., 0., 0.],
       [0., 0., 0., 0., 0., 0., 5., 0., 0., 3., 1., 2., 0., 0., 4., 0.],
       [0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 3., 1., 0., 0., 0., 4.],
       [0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 1., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0.,